In [1]:
import pandas as pd
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
import jieba

Using TensorFlow backend.


In [2]:
data_train=pd.read_csv(r'E:\baidumatch\sentiment\data\data_train.csv',encoding='gbk',delimiter="\t",header=None,index_col=0)
data_test=pd.read_csv(r'E:\baidumatch\sentiment\data\data_test.csv',encoding='gbk',delimiter='\t',header=None,index_col=0)

In [3]:
data_train
data_train.columns=['a','b','c']
#data_test

In [4]:
data_train.iloc[:,0]
dict={}
for i in data_train.iloc[:,0]:
    if i in dict:
        dict[i]+=1
    else:
        dict[i]=1
print(dict)

{'金融服务': 14344, '医疗服务': 20312, '旅游住宿': 18935, '食品餐饮': 13655, '物流快递': 14779}


In [5]:
industry_dict={'食品餐饮':0,'医疗服务':1,'金融服务':2,'旅游住宿':3,'物流快递':4}

In [6]:
data_train.iloc[:,0]=data_train.iloc[:,0].map(industry_dict)#注意：不要写成data_train.iloc[:,0].map(lambda x:industry_dict),这样就将每一行都映射为industry_dict了

In [7]:
pinglun=data_train['b'].values.tolist()

In [8]:
contents_cut=[]
for i in pinglun:
    sentence_cut=jieba.lcut(str(i))
    if sentence_cut!='\r\n': #有内容且不是换行符
        contents_cut.append(sentence_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\YWZQ\AppData\Local\Temp\jieba.cache
Loading model cost 1.074 seconds.
Prefix dict has been built succesfully.


In [9]:
stopwords=pd.read_csv(r'F:\tang_data\text_analy\stopwords.txt',index_col=False,sep='\t',quoting=3,names=['stopword'],encoding='utf-8')

In [10]:
def drop_stopwords(contents,stopwords):
    contents_clean=[]
    all_clean_words=[]
    for line in contents:
        line_clean=[]
        for i in line:
            if i in stopwords or i=='[' or i==']':
                continue
            line_clean.append(i)
            all_clean_words.append(i)
            #line_clean_str=' '.join(line_clean_str)
        contents_clean.append(line_clean)
    return contents_clean,all_clean_words
stopwords=stopwords.stopword.values.tolist()
contents_clean,all_clean_words=drop_stopwords(contents_cut,stopwords)  #all_clean_words主要用来词云显示，对于算法模型无用
df_contents_clean=pd.DataFrame({'contents_cut_clean':contents_clean})
df_contents_clean.head()

,contents_cut_clean
0,"[买, 这套, 系统, 本来, 做, 公司, 公众, 号, 第三方, 平台, 代, 运营, ..."
1,"[烤鸭, 不错, 菜, 没什么]"
2,"[看不懂, 操作, 详细, 标明]"
3,"[越来越, 不好, 菜品, 少, 服务]"
4,[吃]


In [11]:
df_contents_clean.index = range(1,len(df_contents_clean) + 1)

In [12]:
df_contents_clean

,contents_cut_clean
1,"[买, 这套, 系统, 本来, 做, 公司, 公众, 号, 第三方, 平台, 代, 运营, ..."
2,"[烤鸭, 不错, 菜, 没什么]"
3,"[看不懂, 操作, 详细, 标明]"
4,"[越来越, 不好, 菜品, 少, 服务]"
5,[吃]
6,"[这家, 公司, 合作, 系统, 功能, 强大, 运行, 流畅, 不错]"
7,"[点, 中辣, 平时, 店里, 吃, 辣, 叫外卖, 炒, 吃花, 时间, 长, 辣味, 缘..."
8,"[专业, 用心, 做, 出品, 好评]"
9,"[不错, 下次, 机会, 还来, 哈哈哈, 哈哈哈]"
10,"[不错, 品种齐全, 上菜, 很快, 味道, 不错, 服务态度]"


In [13]:
data_train.b=df_contents_clean.contents_cut_clean

In [14]:
data_train

,a,b,c
0,,,
1,0,"[买, 这套, 系统, 本来, 做, 公司, 公众, 号, 第三方, 平台, 代, 运营, ...",2
2,0,"[烤鸭, 不错, 菜, 没什么]",1
3,0,"[看不懂, 操作, 详细, 标明]",0
4,0,"[越来越, 不好, 菜品, 少, 服务]",0
5,0,[吃],1
6,0,"[这家, 公司, 合作, 系统, 功能, 强大, 运行, 流畅, 不错]",2
7,0,"[点, 中辣, 平时, 店里, 吃, 辣, 叫外卖, 炒, 吃花, 时间, 长, 辣味, 缘...",2
8,0,"[专业, 用心, 做, 出品, 好评]",2
9,0,"[不错, 下次, 机会, 还来, 哈哈哈, 哈哈哈]",2


In [15]:
data_train.b=data_train.b.map(lambda x:' '.join(x))
data_train

,a,b,c
0,,,
1,0,买 这套 系统 本来 做 公司 公众 号 第三方 平台 代 运营 没想到 app 方便管理 ...,2
2,0,烤鸭 不错 菜 没什么,1
3,0,看不懂 操作 详细 标明,0
4,0,越来越 不好 菜品 少 服务,0
5,0,吃,1
6,0,这家 公司 合作 系统 功能 强大 运行 流畅 不错,2
7,0,点 中辣 平时 店里 吃 辣 叫外卖 炒 吃花 时间 长 辣味 缘故 味道 下次 叫外卖 点...,2
8,0,专业 用心 做 出品 好评,2
9,0,不错 下次 机会 还来 哈哈哈 哈哈哈,2


In [16]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

In [38]:
token_0=Tokenizer(num_words=3000)
token_1=Tokenizer(num_words=3000)
token_2=Tokenizer(num_words=3000)
token_3=Tokenizer(num_words=3000)
token_4=Tokenizer(num_words=3000)


train_0=data_train.loc[data_train['a'] == 0]
token_0.fit_on_texts(train_0.b)
train_0_seq=token_0.texts_to_sequences(train_0.b)
x_0=sequence.pad_sequences(train_0_seq,15)
y_0=train_0.c
y_0_onehot=np_utils.to_categorical(y_0)


train_1=data_train.loc[data_train['a'] == 1]
token_1.fit_on_texts(train_1.b)
train_1_seq=token_1.texts_to_sequences(train_1.b)
x_1=sequence.pad_sequences(train_1_seq,15)
y_1=train_1.c
y_1_onehot=np_utils.to_categorical(y_1)


train_2=data_train.loc[data_train['a'] == 2]
token_2.fit_on_texts(train_2.b)
train_2_seq=token_2.texts_to_sequences(train_2.b)
x_2=sequence.pad_sequences(train_2_seq,15)
y_2=train_2.c
y_2_onehot=np_utils.to_categorical(y_2)


train_3=data_train.loc[data_train['a'] == 3]
token_3.fit_on_texts(train_3.b)
train_3_seq=token_3.texts_to_sequences(train_3.b)
x_3=sequence.pad_sequences(train_3_seq,15)
y_3=train_3.c
y_3_onehot=np_utils.to_categorical(y_3)


train_4=data_train.loc[data_train['a'] == 4]
token_4.fit_on_texts(train_4.b)
train_4_seq=token_4.texts_to_sequences(train_4.b)
x_4=sequence.pad_sequences(train_4_seq,15)
y_4=train_4.c
y_4_onehot=np_utils.to_categorical(y_4)

In [21]:
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
#from keras.layers.recurrent import SimpleRNN
from keras.layers.recurrent import LSTM

In [25]:
model=Sequential()
model.add(Embedding(output_dim=32,input_dim=3000,input_length=15))
model.add(Dropout(0.2))

In [26]:
model.add(LSTM(32))
model.add(Dense(units=250,activation='relu'))
model.add(Dropout(0.35))

In [27]:
model.add(Dense(units=3,activation='softmax'))

In [45]:
import keras
adam=keras.optimizers.Adamax(lr=0.008)

In [46]:
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['categorical_accuracy'])
#help(model.compile)

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 15, 32)            96000     
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 32)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_2 (Dense)              (None, 250)               8250      
_________________________________________________________________
dropout_5 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 753       
Total params: 113,323
Trainable params: 113,323
Non-trainable params: 0
_________________________________________________________________


In [48]:
train_history=model.fit(x_0,y_0_onehot,batch_size=100,epochs=60,verbose=2,validation_split=0.2)

Train on 10924 samples, validate on 2731 samples
Epoch 1/60
 - 3s - loss: 0.0739 - categorical_accuracy: 0.9724 - val_loss: 1.5467 - val_categorical_accuracy: 0.7708
Epoch 2/60
 - 2s - loss: 0.0616 - categorical_accuracy: 0.9747 - val_loss: 1.5895 - val_categorical_accuracy: 0.7671
Epoch 3/60
 - 2s - loss: 0.0618 - categorical_accuracy: 0.9740 - val_loss: 1.5968 - val_categorical_accuracy: 0.7704
Epoch 4/60
 - 2s - loss: 0.0571 - categorical_accuracy: 0.9765 - val_loss: 1.5926 - val_categorical_accuracy: 0.7682
Epoch 5/60
 - 2s - loss: 0.0548 - categorical_accuracy: 0.9787 - val_loss: 1.6860 - val_categorical_accuracy: 0.7689
Epoch 6/60
 - 2s - loss: 0.0581 - categorical_accuracy: 0.9782 - val_loss: 1.6069 - val_categorical_accuracy: 0.7671
Epoch 7/60
 - 2s - loss: 0.0578 - categorical_accuracy: 0.9786 - val_loss: 1.6182 - val_categorical_accuracy: 0.7609
Epoch 8/60
 - 2s - loss: 0.0562 - categorical_accuracy: 0.9778 - val_loss: 1.6135 - val_categorical_accuracy: 0.7679
Epoch 9/60
 - 2

In [49]:
train_history=model.fit(x_1,y_1_onehot,batch_size=100,epochs=60,verbose=2,validation_split=0.2)

Train on 16249 samples, validate on 4063 samples
Epoch 1/60
 - 2s - loss: 0.5277 - categorical_accuracy: 0.9212 - val_loss: 0.3122 - val_categorical_accuracy: 0.9271
Epoch 2/60
 - 2s - loss: 0.2696 - categorical_accuracy: 0.9338 - val_loss: 0.2716 - val_categorical_accuracy: 0.9279
Epoch 3/60
 - 2s - loss: 0.2245 - categorical_accuracy: 0.9360 - val_loss: 0.2543 - val_categorical_accuracy: 0.9291
Epoch 4/60
 - 2s - loss: 0.1908 - categorical_accuracy: 0.9398 - val_loss: 0.2361 - val_categorical_accuracy: 0.9328
Epoch 5/60
 - 2s - loss: 0.1590 - categorical_accuracy: 0.9478 - val_loss: 0.2234 - val_categorical_accuracy: 0.9355
Epoch 6/60
 - 2s - loss: 0.1376 - categorical_accuracy: 0.9536 - val_loss: 0.2237 - val_categorical_accuracy: 0.9392
Epoch 7/60
 - 2s - loss: 0.1185 - categorical_accuracy: 0.9616 - val_loss: 0.2378 - val_categorical_accuracy: 0.9427
Epoch 8/60
 - 2s - loss: 0.1044 - categorical_accuracy: 0.9648 - val_loss: 0.2319 - val_categorical_accuracy: 0.9441
Epoch 9/60
 - 2

KeyboardInterrupt: 

In [ ]:
train_history=model.fit(x_2,y_2_onehot,batch_size=100,epochs=60,verbose=2,validation_split=0.2)

In [ ]:
train_history=model.fit(x_3,y_3_onehot,batch_size=100,epochs=60,verbose=2,validation_split=0.2)

In [ ]:
train_history=model.fit(x_4,y_4_onehot,batch_size=100,epochs=60,verbose=2,validation_split=0.2)